In [31]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.cuda import amp
import timm
from pytorch_toolbelt.inference import tta
from pytorch_toolbelt import losses as L

import random
from tqdm.auto import tqdm
from PIL import Image
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import os
import json
import pandas as pd
import segmentation_models_pytorch as smp
from monai.inferers import sliding_window_inference
import shutil
from tifffile import imread
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [32]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape).T

def read_rgb_img(img_path):
#     img_path= img_path.replace('train_images', 'train_images_domain_shift')
    img= imread(img_path)
    orign_shape= img.shape[:2][::-1]
    return img, orign_shape

def get_test_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ])

class Customize_Dataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.image_path = df['image_path'].values
        self.transforms = transforms
    
    def __getitem__(self, index):
        img_path = self.image_path[index]
        img, ori_shape= read_rgb_img(img_path)
        
        ## scale adjust
        img_size= min(img.shape[0], img.shape[1])
        img_size= img_size//CFG['img_scale']
        img_size= CFG['window_size'] if img_size < CFG['window_size'] else img_size
        img= cv2.resize(img, (img_size, img_size))
        
        img = self.transforms(image=img)["image"]
        return {
            'img_path': img_path,
            'image': torch.tensor(img/255, dtype=torch.float32),
            'ori_shape': torch.tensor(ori_shape),
        }
    
    def __len__(self):
        return len(self.df)

# CFG

In [33]:
shutil.rmtree('valid_temp')
os.mkdir('valid_temp')
os.mkdir('valid_temp/gt')
os.mkdir('valid_temp/pt')

In [34]:
CFG= {
    'fold': 3,
    'img_scale': 4,
    'window_size': 768,
    'TTA': False,
    'model': None,
    'show_result': False,
}
# CFG['model']= f"./train_model/model_cv{CFG['fold']}_best.pth"
# CFG['model']= f"./train_model/model_cv{CFG['fold']}_ep81.pth"
CFG['model']= f"./test_model/effb7_w768_0.54/model_cv{CFG['fold']}_best.pth"

CFG['model']= [torch.load(CFG['model'], map_location= 'cuda:0')]

# Prepare Dataset

In [35]:
df= pd.read_csv('./Data/train.csv')

valid_df= df[df['fold']==CFG['fold']].reset_index(drop=True)
print(f'valid dataset: {len(valid_df)}')

valid_dataset= Customize_Dataset(valid_df, get_test_transform())
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0)
valid_df.head()

valid dataset: 70


,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex,image_path,fold
0,10044,prostate,HPA,3000,3000,0.4,4,1459676 77 1462675 82 1465674 87 1468673 92 14...,37.0,Male,Data/train_images/10044.tiff,3
1,10274,prostate,HPA,3000,3000,0.4,4,715707 2 718705 8 721703 11 724701 18 727692 3...,76.0,Male,Data/train_images/10274.tiff,3
2,11645,spleen,HPA,2867,2867,0.4,4,1971766 32 1974633 34 1977499 36 1980366 37 19...,74.0,Female,Data/train_images/11645.tiff,3
3,11890,largeintestine,HPA,3000,3000,0.4,4,1297230 32 1300222 46 1303215 60 1306213 63 13...,79.0,Female,Data/train_images/11890.tiff,3
4,1220,lung,HPA,3000,3000,0.4,4,1429797 16 1432793 21 1435791 23 1438789 27 14...,59.0,Male,Data/train_images/1220.tiff,3


# Inference

In [36]:
def inference(model, img, ori_shape):
    img= img.cuda()
    for i, m in enumerate(model):
        with torch.no_grad():
            m.eval()
            if CFG['TTA']:
                pred_1= m(img)[0]
                pred_2= m(img.flip(-1))[0].flip(-1)
                pred_3= m(img.flip(-2))[0].flip(-2)
                pred= (pred_1 + pred_2 + pred_3) / 3
            else:
                pred= sliding_window_inference(img, 
                                               (CFG['window_size'], CFG['window_size']), 
                                               sw_batch_size= 2, 
                                               predictor= m,
                                               mode= 'gaussian',
                                               overlap= 0.25)[0]
                
        if i==0: preds= pred
        else: preds+= pred
    pred= preds/len(model)
    pred= pred.sigmoid().cpu().permute(1,2,0).numpy()
    pred= cv2.resize(pred, tuple(ori_shape))
    return pred

In [37]:
for i, data in enumerate(tqdm(valid_loader)):
    for j in range(len(data['image'])):
        img_path= data['img_path'][j]
        img= data['image'][j]
        ori_shape= data['ori_shape'][j]
        
        img_size= Image.open(img_path).size[::-1]
        id_= img_path.split('/')[-1].split('.')[0]
        rle= valid_df.loc[valid_df['id']==int(id_), 'rle'].values[0]
        gt_mask= rle_decode(rle, img_size)
        
        ## inference
        img= torch.unsqueeze(img, dim= 0)
        pred_mask= inference(CFG['model'], img, ori_shape.numpy())
    
        pred_mask= pred_mask*255
        im= Image.fromarray(pred_mask.astype(np.uint8))
        im.save(f'valid_temp/pt/{id_}.png')
        gt_mask= gt_mask*255
        im= Image.fromarray(gt_mask.astype(np.uint8))
        im.save(f'valid_temp/gt/{id_}.png')
        
        if CFG['show_result']:
            ## show result
            plt.figure(figsize=(15,15))
            plt.subplot(1,4,1)
            img= np.array(Image.open(img_path))
            plt.title('image', color= 'b')
            plt.imshow(img)

            plt.subplot(1,4,2)
            plt.title('predict_mask', color= 'b')
            plt.imshow(pred_mask)

            pred_mask= cv2.cvtColor(pred_mask, cv2.COLOR_GRAY2RGB)
            pred_mask[:,:,1:]= 0
            mix_img= (img*0.5).astype(np.uint8) + (pred_mask*0.5).astype(np.uint8)
            plt.subplot(1,4,3)
            plt.title('mix_iamge', color= 'b')
            plt.imshow(mix_img)

            gt_mask= cv2.cvtColor(gt_mask, cv2.COLOR_GRAY2RGB)
            gt_mask[:,:,1:]= 0
            mix_img= (img*0.5).astype(np.uint8) + (gt_mask*0.5).astype(np.uint8)
            plt.subplot(1,4,4)
            plt.title('gt_mask', color= 'b')
            plt.imshow(mix_img)
            plt.show()

  0%|          | 0/70 [00:00<?, ?it/s]

# Metric

In [ ]:
gt= glob.glob('valid_temp/gt/**.png')
pt= glob.glob('valid_temp/pt/**.png')

from pytorch_toolbelt import losses as L
dice_loss= L.DiceLoss(mode= 'binary', from_logits=False)

def evaluate(thr= 0.5):
    dice_organ= {
        'prostate': [],
        'spleen': [],
        'lung': [],
        'kidney': [],
        'largeintestine': [],
    }
    dices= []
    for i in tqdm(range(len(gt))):
        id_= pt[i].split('\\')[-1].split('.')[0]
        organ= df[df['id']==int(id_)]['organ'].values[0]

        pt_mask= np.array(Image.open(pt[i]).convert('L'))/255
        pt_mask= np.expand_dims(pt_mask, axis=0)
        pt_mask[pt_mask>=thr]= 1
        pt_mask[pt_mask<thr]= 0
        gt_mask= np.array(Image.open(gt[i]).convert('L'))/255
        gt_mask= np.expand_dims(gt_mask, axis=0)

        loss= dice_loss( torch.tensor(pt_mask), torch.tensor(gt_mask) )
        score= 1-loss
        dices.append(score)

        dice_organ[organ].append(score)

    for key in dice_organ.keys():
        dice_organ[key]= round( np.mean(dice_organ[key]), 3)
        
    return np.mean(dices), dice_organ

scores= []
for thr in range(1, 10):
    thr/=10
    print(f'thr= {thr}')
    dice, organ_dice= evaluate(thr)
    scores.append(dice)
    if thr==0.5:
        print(f'dice_score: {np.mean(dice)}')
        print(f'dice_organ:')
        display(organ_dice)
        
avg_dice= []
for (key, value) in organ_dice.items():
    avg_dice.append(value)
print(f'avg_dice: {np.mean(avg_dice)}')

thr= 0.1


  0%|          | 0/70 [00:00<?, ?it/s]

In [ ]:
plt.plot(scores)
plt.show()

In [ ]:
# cv_score_thr= []
cv_score_thr.append(scores)

In [ ]:
plt.plot(cv_score_thr[0])
plt.plot(cv_score_thr[1])
plt.plot(cv_score_thr[2])
plt.plot(cv_score_thr[3])
plt.plot(cv_score_thr[4])

In [ ]:
hpa_lb= 0.22
hpa_lb_norm= hpa_lb*(291/81)
print(f'hpa_norm: {hpa_lb_norm}')

hubmap_lb= 0.54
hubmap_lb_norm= hubmap_lb*(291/210)
print(f'hubmap_norm: {hubmap_lb_norm}')